# Interactive Widgets

### By Carlos Santillán

### Date: May 2020

## 1. Introduction

It is 1st necessary to install *ipywidgets*, this is done in the command line as follows:

1. pip install ipywidgets

2. jupyter nbextension enable --py widgetsnbextension

**Following are the necessary imports along the way:**

In [84]:
import ipywidgets as widgets           ### widgets
from IPython.display import display    ### display widgets 
import seaborn as sns                  ### statistical plots
import matplotlib.pyplot as plt        ### plots

C:\Users\SANTICA4\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### 1.2 Adding a Widget

First we import the module:

In [2]:
import ipywidgets as widgets

A useful one is a slider, for this one we can define:

- minimum value

- maximum value

- interval size

- description

- initial value

In [3]:
widgets.IntSlider( 
    min = 0,
    max = 10,
    step = 1,
    description = 'Slider:',
    value = 3
)

IntSlider(value=3, description='Slider:', max=10)

### 1.3 Displaying Widgets

This is done by the *display()* function, which renders a widget in an input cell in our notebook.

First we need to import *display*:

In [4]:
from IPython.display import display

Then we need to pass pur previously created widget as a parameter in the *display()* dunction:

In [5]:
slider = widgets.IntSlider( 
    min = 0,
    max = 10,
    step = 1,
    description = 'Slider:',
    value = 3
)

display(slider)

IntSlider(value=3, description='Slider:', max=10)

### 1.4 Getting & Setting Widget´s value

In order to read the value of a widget, we need to *query* its value property.

In [7]:
slider.value

8

Similarly, we can set its value:

In [10]:
slider.value = 1

### 1.5 Linking 2 Widgets

We can synchronise the values of 2 widgets by using the *jslink()* function:

In [11]:
text = widgets.IntText()

display(slider, text)

IntSlider(value=1, description='Slider:', max=10)

IntText(value=0)

In [13]:
widgets.jslink((slider, 'value'), (text, 'value'))

display(slider, text)

IntSlider(value=6, description='Slider:', max=10)

IntText(value=6)

### 1.5 Widget list

We can get a full list of widgets in the documentation, or by runnning the following:

In [14]:
print(dir(widgets))

['Accordion', 'AppLayout', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Combobox', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FileUpload', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'GridBox', 'GridspecLayout', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'TwoByTwoLayout', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_vers

## 2. Handling Widget Events

Widgets can respond to events which are raised when a user interacts with them. 

A simple example is clicking on a button, then we expect an action to take place.

Depending on its specific features, each widget exposes different events. An **event handler** will be executed every time the event is fired.

*Event handler is a callback function in response to an evetn, that operates asynchronously and handles the inputs received*

So now we´ll create a simple button called *btn*. The *on_click* method is invoked when the button is clicked.

Our event handler *btn_eventhandler* will print a short message with the button´s caption, please note that the input argument of the handler, *obj*, is the butthon object itself which allows us to access its properties.

To bind the event with the handler, we assign the latter to the button´s *on_click* method:

In [15]:
btn = widgets.Button(description = 'BI team')
display(btn)

def btn_eventhandler(obj):
    '''
    DESC.: prints message with button´s caption
    INPUT: widget object
    OUTPUT: Message
    '''
    print('Hello from the {} button!'.format(obj.description))
    
btn.on_click(btn_eventhandler)

Button(description='BI team', style=ButtonStyle())

Hello from the BI team button!
Hello from the BI team button!


Notice that the output appears in the same cell as the button itself. We can make this more flexible

## 3. Controlling Widget Output

We can use our widgets to **control a dataframe**. 

The sample dataset chose is *Number of International Visitors to London* which shows totals of London´s visitors regarding nights, visits and spend; broken down by year, quarter, purpose, duration, mode and country.

First we get the data and load it into a dataframe:

In [22]:
import pandas as pd
import numpy as np

url = "C:/Users/SANTICA4/OneDrive - Novartis Pharma AG/Desktop/Personal/london_data.csv"

df_london = pd.read_csv(url)

In [23]:
df_london.head()

,year,quarter,market,dur_stay,mode,purpose,area,Visits (000s),Spend (pounds),Nights (000s),sample
0,2002,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,3.572186,0.969138,6.954456,5
1,2002,January-March,Belgium,1-3 nights,Air,Business,LONDON,9.284226,2.399577,12.604959,19
2,2002,January-March,Belgium,1-3 nights,Air,VFR,LONDON,0.877182,0.089833,2.153128,3
3,2002,January-March,Belgium,1-3 nights,Air,Miscellaneous,LONDON,0.163874,0.010160,0.163874,1
4,2002,January-March,Belgium,1-3 nights,Sea,Business,LONDON,1.648670,0.016789,1.650300,1


Now suppose we want to filter the dataframe by year. We will first define a dropdown and populate it with the list of unique year values.

In order to do this, we will create a generic function *unique_sorted_values_plus_ALL* which will find the unique values, sort them and then add the ALL item at the start so that the user can remove the filter:

In [24]:
ALL = "ALL"

def unique_sorted_values_plus_ALL(array):
    '''
    DESC.: takes an array of values, finds unique items, sorts them and then adds the value ALL
    INPUT: array of values
    OUTPUT: array w/ unique, sorted values plus ALL
    '''
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    
    return unique

Now we initialize the dropdown:

In [25]:
dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year)
)

The dropdown widget exposes the *observe* method, which takes a function that will be invoked when the value of the dropdown changes.

As such, we will create the observer handler to filter the dataframe by the selected values - note that the input argument of the handler, *change*, contains information about the changes that toook place, which allows us to access the *new* value *change.new*.

If the new value is ALL, we remove de filter, otherwise we apply it:

In [26]:
def dropdown_year_eventhandler(change):
    '''
    DESC.: displays dataframe applying filter
    INPUT: change in the value of filter
    OUTPUT: dataframe with applied filter according to value
    '''
    if (change.new == ALL):
        display(df_london)
    else:
        display(df_london[df_london.year == change.new])

We will then bind the handler to the dropdown:

In [27]:
dropdown_year.observe(
    dropdown_year_eventhandler, 
    names = 'value')

In [29]:
display(dropdown_year)

Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

,year,quarter,market,dur_stay,mode,purpose,area,Visits (000s),Spend (pounds),Nights (000s),sample
33865,2012,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,1.476037,0.517613,3.713900,2
33866,2012,January-March,Belgium,1-3 nights,Air,Business,LONDON,5.512713,2.180262,6.785043,8
33867,2012,January-March,Belgium,1-3 nights,Air,Miscellaneous,LONDON,1.517074,0.270735,1.517074,3
33868,2012,January-March,Belgium,1-3 nights,Sea,Holiday,LONDON,1.876546,0.549828,4.691370,2
33869,2012,January-March,Belgium,1-3 nights,Sea,Business,LONDON,3.022950,0.105486,5.131900,3
...,...,...,...,...,...,...,...,...,...,...,...
37402,2012,October-December,Other Africa,8-14 nights,Air,VFR,LONDON,2.090298,1.165625,18.254010,5
37403,2012,October-December,Other Africa,8-14 nights,Air,Miscellaneous,LONDON,1.042989,1.334698,12.405450,2
37404,2012,October-December,Other Africa,15+ nights,Air,Holiday,LONDON,1.546761,1.929341,51.009061,4
37405,2012,October-December,Other Africa,15+ nights,Air,Business,LONDON,1.467260,0.839446,26.794339,2


Our dropdown behaves as expected!

But, there is an issue: the output of all queries is **accumulating** in this very same cell; i.e. if we select a new year from the dropdown, a new dataframe will render underneath the first one, on the same cell.

The truly desired behavior though, is to **refresh** the contents of the dataframe each time.

### 3.1 Capturing the Widget Output

The solution to this issue is to capture the cell output in a special kind of widget, namely *Output*, and then display it in another cell.

So let´s slightly tweak the code to:

- create a new instance of *Output*:

In [30]:
output_year = widgets.Output()

- call the *clear_output* method within the event handler to clear the previous selection on each iteration and capture the output of the dataframe in a *with* block:

In [31]:
def dropdown_year_eventhandler(change):
    '''
    DESC.: displays dataframe applying filter
    INPUT: change in the value of filter
    OUTPUT: dataframe with applied filter according to value
    '''
    output_year.clear_output()
    
    with output_year:
        display(df_london[df_london.year == change.new])
    

We will then display the output in a new cell:

In [32]:
display(output_year)

Output()

So, our final code is as follows:

In [35]:
dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year))

output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    '''
    DESC.: displays dataframe applying filter
    INPUT: change in the value of filter
    OUTPUT: dataframe with applied filter according to value
    '''
    output_year.clear_output()
    
    with output_year:
        if change.new == ALL:
            display(df_london)
        else:
            display(df_london[df_london.year == change.new])
            
dropdown_year.observe(dropdown_year_eventhandler,
                     names = "value")

display(dropdown_year)


Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

In [36]:
display(output_year)

Output()

As you can see, the output is rendered in a new cell and the filtering is working as expected!

## 4. Linking Widget Outputs

Continuing with the previous example, let´s assume we would also like to filter by *purpose* too.

We can go ahead and add a similar dropdown, but you will realize that the dataframe only responds to the filter by the dropdown which has recently changed, not both.

What we need to do is to **link** the 2 together so ir can work on both values (i.e. year and purpose).

First we need a common output for both dropdowns:

In [56]:
output = widgets.Output()

Here are the 2 dropdowns:

In [57]:
dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year))

dropdown_purpose = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.purpose))


Then we create a new function: *common_filtering*, that will be called by both event handlers. This function will apply a filter on the dataframe for both, year AND purpose.

We´re clearing the output, then we check if any of the values is ALL, in which case we consider that the respective filter is removed. When both filters are present, in the else statement we apply *&* operation in both filters.

Finally we capture the output:

In [58]:
def common_filtering(year, purpose):
    '''
    DESC.: applies both filters to dataframe
    INPUT: year, as variable in dataframe
    INPUT: purpose, as variable in dataframe
    OUTPUT:
    '''
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif year == ALL:
        common_filter = df_london[df_london.purpose == purpose]
    elif purpose == ALL:
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

We ammend the event handlers to call *common_filtering* function and pass the *change.new* value as well as the current *value* of the other dropdown:

In [65]:
def dropdown_year_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(change.new, dropdown_purpose.value)

def dropdown_purpose_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, change.new)

We bind the handlers to the dropdowns and that´s it!

In [66]:
dropdown_year.observe(dropdown_year_eventhandler,
                     names = 'value')

dropdown_purpose.observe(dropdown_purpose_eventhandler,
                        names = 'value')

display(dropdown_year)
display(dropdown_purpose)

Dropdown(index=15, options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '20…

Dropdown(index=3, options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='Miscellaneou…

Here's all the code in one block:

In [68]:
output = widgets.Output()

dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year))

dropdown_purpose = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.purpose))

def common_filtering(year, purpose):
    '''
    DESC.: applies both filters to dataframe
    INPUT: year, as variable in dataframe
    INPUT: purpose, as variable in dataframe
    OUTPUT:
    '''
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif year == ALL:
        common_filter = df_london[df_london.purpose == purpose]
    elif purpose == ALL:
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

def dropdown_year_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(change.new, dropdown_purpose.value)

def dropdown_purpose_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler,
                     names = 'value')

dropdown_purpose.observe(dropdown_purpose_eventhandler,
                        names = 'value')

display(dropdown_year)
display(dropdown_purpose)

Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

In [71]:
display(output)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '       year           quarter        m…

In [72]:
output = widgets.Output()
plot_output = widgets.Output()



## 5. Creating a Dashboard

We can carry on our exercise by colouring the numeric values based on a user selected value.

A useful numeric widget is the *BoundedFloatText*; we can give it a *min*, *max*, initial *value* and incremental *step*:

In [73]:
bounded_num = widgets.BoundedFloatText(
    min = 0, 
    max = 100000, 
    value = 5, 
    step = 1)

In order to color the dataframe cells, we will define this function:

In [74]:
def color_ge_value(value, comparison):
    '''
    DESC.: Colors cells according to values
    INPUT: value
    INPUT: comparison
    OUTPUT: Colored cells in dataframe
    '''
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'


Now we can minimally ammend the *common_filtering* function to:

- add a new *num* parameter

- apply the styling by calling the *color_ge_value* function for the 3 numeric columns


In [ ]:
def common_filtering(year, purpose, num):

In [ ]:
with output:
    display(common_filter.style.applymap(
                            lambda x: color_ge_value(x, num),
                            subset = ['visits', 'spend', 'nights']))

Also, the existing event-handlers need to me adjusted to pass the *bounded_num.value*:

In [ ]:
def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,
                     bounded_num.value)

def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new, 
                     bounded_num.value)

And finally we can plug-in the event-handler of the new widget:

In [ ]:
def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, 
                     change.new)
    
bounded_num.observe(bounded_num_eventhandler, names='value')

So, in a single code we get:

In [81]:
output = widgets.Output()

dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year))

dropdown_purpose = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.purpose))

bounded_num = widgets.BoundedFloatText(
    min = 0, 
    max = 100000, 
    value = 5, step = 1, 
    description = 'Number:')

def common_filtering(year, purpose, num):
    '''
    DESC.: applies both filters to dataframe
    INPUT: year, as variable in dataframe
    INPUT: purpose, as variable in dataframe
    INPUT: num, as a number to be displayed
    OUTPUT:
    '''
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif year == ALL:
        common_filter = df_london[df_london.purpose == purpose]
    elif purpose == ALL:
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(
            lambda x: color_ge_value(x, num),
            subset = ['Visits (000s)', 'Spend (pounds)', 'Nights (000s)']))

def dropdown_year_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)

def dropdown_purpose_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, change.new, bounded_num.value)
    
def bounded_num_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

dropdown_year.observe(dropdown_year_eventhandler,
                     names = 'value')

dropdown_purpose.observe(dropdown_purpose_eventhandler,
                        names = 'value')

bounded_num.observe(bounded_num_eventhandler,
                    names = 'value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

BoundedFloatText(value=5.0, description='Number:', max=100000.0, step=1.0)

In [83]:
display(output)

Output()

### 5.1 Plotting

Now, we will plot the density of the number of visits, which in *seaborn* can be done with KDE (Kernel Density Estimation).

First we import the libraries:

In [85]:
import seaborn as sns
import matplotlib.pyplot as plt

We can then store the plot in a new output variable:

In [86]:
plot_output = widgets.Output()

Again, we need to ammend the *common_filtering* function to plot the new diagram.

- First, we clear the output

In [ ]:
plot_output.clear_output()

- Then we call the *kdeplot* method of seaborn by pasing the number of visits:

In [ ]:
with plot_output:
    sns.kdeplot(common_filter['Visits (000s)'],
                shade = True)
    plt.show

- and lastly, we display the outputs in a new cell:

In [ ]:
display(output)
display(plot_output)

In a snippet of code:

In [87]:
output = widgets.Output()
plot_output = widgets.Output()

dropdown_year = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.year))

dropdown_purpose = widgets.Dropdown(
    options = unique_sorted_values_plus_ALL(df_london.purpose))

bounded_num = widgets.BoundedFloatText(
    min = 0, 
    max = 100000, 
    value = 5, step = 1, 
    description = 'Number:')

def common_filtering(year, purpose, num):
    '''
    DESC.: applies both filters to dataframe
    INPUT: year, as variable in dataframe
    INPUT: purpose, as variable in dataframe
    INPUT: num, as a number to be displayed
    OUTPUT:
    '''
    output.clear_output()
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif year == ALL:
        common_filter = df_london[df_london.purpose == purpose]
    elif purpose == ALL:
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(
            lambda x: color_ge_value(x, num),
            subset = ['Visits (000s)', 'Spend (pounds)', 'Nights (000s)']))

    with plot_output:
        sns.kdeplot(common_filter['Visits (000s)'],
                    shade = True)
        plt.show()
    
def dropdown_year_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(change.new, dropdown_purpose.value, bounded_num.value)

def dropdown_purpose_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, change.new, bounded_num.value)
    
def bounded_num_eventhandler(change):
    '''
    DESC.:
    INPUT:
    OUTPUT:
    '''
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)

    
dropdown_year.observe(dropdown_year_eventhandler,
                     names = 'value')

dropdown_purpose.observe(dropdown_purpose_eventhandler,
                        names = 'value')

bounded_num.observe(bounded_num_eventhandler,
                    names = 'value')

display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

Dropdown(options=('ALL', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012…

Dropdown(options=('ALL', 'Business', 'Holiday', 'Miscellaneous', 'Study', 'VFR'), value='ALL')

BoundedFloatText(value=5.0, description='Number:', max=100000.0, step=1.0)

In [88]:
display(output)
display(plot_output)

Output()

Output()

## 6. Dashboard Layout

We can optimize the look of our dashboard by arranging the input widgets horizontally.

The *HBox* will add widgets to it one at a time from left-to-right

In [90]:
input_widgets = widgets.HBox(
    [dropdown_year, dropdown_purpose, bounded_num])

display(input_widgets)

We then can create a container for the output and *Tab* is great for this. The 1st tab will host the dataframe and the 2nd one the graph:


In [91]:
tab = widgets.Tab([output, plot_output])
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')

display(tab)

And finally we can stack the input widgets and the tab on top of each other with a *VBox*:

In [92]:
dashboard = widgets.VBox([input_widgets, tab])

display(dashboard)

Notice that it feels a bit jammed, so as a last step, we willpolish our dashboard by adding some space. We will define a *Layout* giving 50px margin between the items.

In [93]:
item_layout = widgets.Layout(margin = '0 0 50px 0')

And we will call this layout for each item:

In [97]:
input_widgets = widgets.HBox(
    [dropdown_year, dropdown_purpose, bounded_num],
    layout = item_layout)

tab = widgets.Tab(
    [output, plot_output],
    layout = item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')

And so our final dashboard looks like this:

In [98]:
dashboard = widgets.VBox([input_widgets, tab])

display(dashboard)